In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from shapely.geometry import Point

import contextily as ctx
#from geopy.distance import geodesic
from datetime import datetime


In [ ]:
filename_DD = '../data/nextbike/trips_Dresden 2025-03-14_19-08-34.pkl'
filename_FB = '../data/nextbike/trips_Freiburg 2025-03-14_19-08-34.pkl'
df_DD = pd.read_pickle(filename_DD)
df_FB = pd.read_pickle(filename_FB)

In [ ]:
df_DD[df_DD.station_number_return.isna()].sample(20)

## illustate all locations of bikes ever

In [ ]:
df_tmp = df_FB
geometry = [Point(xy) for xy in zip(df_tmp.lng_return, df_tmp.lat_return)]
gdf = gpd.GeoDataFrame(df_tmp, geometry=geometry, crs="EPSG:4326") 
# crs is for coordinate reference system

gdf = gdf.to_crs(epsg=3857) 

round_factor = -2
gdf["x_round"] = gdf.geometry.x.round(round_factor)  
gdf["y_round"] = gdf.geometry.y.round(round_factor)

gdf_st = gdf[~gdf.station_number_return.isna()]
gdf_ff = gdf[gdf.station_number_return.isna()]

gdf_st_size = gdf_st.groupby(["x_round", "y_round"]).size().reset_index(name="count")
gdf_ff_size = gdf_ff.groupby(["x_round", "y_round"]).size().reset_index(name="count")

fig, ax = plt.subplots(figsize=(8, 8))
#ax.set_xlim([xmin, xmax])
#ax.set_ylim([ymin, ymax])
gdf_ff_size.plot(kind="scatter", x="x_round", y="y_round", alpha=0.85 ,s=gdf_ff_size["count"] * 0.05, color="cornflowerblue", ax=ax, edgecolors="none", label = "Free-floating bikes")
gdf_st_size.plot.scatter(x="x_round", y="y_round", alpha=0.5 ,s=gdf_st_size["count"] * 0.05, color="firebrick", ax=ax, label="Docked bikes", edgecolors="none")



ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.axis('off')
plt.legend(loc="upper left")
plt.savefig(f'/Users/v.sinichenko/Downloads/plots/ever_returned_bikes_map_FB.png', format='png', bbox_inches='tight')
plt.show()